### Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.integrate import odeint
import pysindy as ps
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import complex_ode
a = np.loadtxt('./a.dat')
print(a.shape)
t = a[:, 0]
r = 9
dt = t[1] - t[0]
if r == 3:
    a_temp = a[:, 1:r]
    a_temp = np.hstack((a_temp, a[:, 9].reshape(len(t), 1)))
    a = a_temp
else:
    a = a[:, 1:r+1]
a_max = max(a[:, :2].flatten())

(3000, 10)


In [2]:
def pg_model(a, L, Q):
    """RHS of POD-Galerkin model, for time integration"""
    return (L @ a) + np.einsum('ijk,j,k->i', Q, a, a)

galerkin3 = sio.loadmat('models/galerkin3.mat')
model3 = lambda a, t: pg_model(a, galerkin3['L'], galerkin3['Q'])

galerkin9 = sio.loadmat('models/galerkin9.mat')
model9 = lambda a, t: pg_model(a, galerkin9['L'], galerkin9['Q'])

# Simulate Galerkin models
t_sim = np.arange(0, 500, dt)

# Generate initial condition from unstable eigenvectors
lamb, Phi = np.linalg.eig(galerkin9['L'])
idx = np.argsort(-np.real(lamb))
lamb, Phi = lamb[idx], Phi[:, idx]
a0 = np.real( 1e-3*Phi[:, :2] @ np.random.random((2)) )

a_galerkin3 = odeint(model3, a0[:3], t_sim)
a_galerkin9 = odeint(model9, a0[:9], t_sim)

In [3]:
## Setup hyperparameters and matrices
Nr = int((r**2 + 3*r)/2.0)
q = 0
constraint_zeros = np.zeros((r + r*(r-1) + int(r*(r-1)*(r-2)/6.0)))
constraint_matrix = np.zeros((r + r*(r-1) + int(r*(r-1)*(r-2)/6.0), r * Nr))

# Set coefficients adorning terms like a_i^3 to zero
for i in range(r):
    constraint_matrix[q, r*(int((r**2+3*r)/2.0)-r) + i*(r+1)] = 1.0
    q = q + 1

# Set coefficients adorning terms like a_ia_j^2 to be antisymmetric
for i in range(r):
    for j in range(i+1, r):
        constraint_matrix[q, r*(int((r**2+3*r)/2.0)-r+j)+i] = 1.0
        constraint_matrix[q, r*(r+j-1)+j+r*int(i*(2*r-i-3)/2.0)] = 1.0
        q = q + 1
for i in range(r):
     for j in range(0, i):
        constraint_matrix[q, r*(int((r**2+3*r)/2.0)-r+j)+i] = 1.0
        constraint_matrix[q, r*(r+i-1)+j+r*int(j*(2*r-j-3)/2.0)] = 1.0
        q = q + 1

# Set coefficients adorning terms like a_ia_ja_k to be antisymmetric
for i in range(r):
    for j in range(i+1, r):
        for k in range(j+1, r):
            constraint_matrix[q, r*(r+k-1)+i+r*int(j*(2*r-j-3)/2.0)] = 1.0
            constraint_matrix[q, r*(r+k-1)+j+r*int(i*(2*r-i-3)/2.0)] = 1.0
            constraint_matrix[q, r*(r+j-1)+k+r*int(i*(2*r-i-3)/2.0)] = 1.0
            q = q + 1
# delta_{il}delta_{jk}
PL_tensor = np.zeros((r, r, r, Nr))
for i in range(r):
    for j in range(r):
        for k in range(r):
            for l in range(Nr):
                if i == l and j == k:
                    PL_tensor[i, j, k, l] = 1.0

# Now symmetrize PL
for i in range(r):
    for j in range(Nr):
        PL_tensor[:, :, i, j] = 0.5 * (PL_tensor[:, :, i, j] + PL_tensor[:, :, i, j].T)

PQ_tensor= np.zeros((r, r, Nr))
for i in range(r):
    # Off diagonal terms
    for j in range(i+1, r):
        PQ_tensor[i, j, int((i+1)/2.0*(2*r-i)) + j - 1 - i] = 1.0

    # diagonal terms
    PQ_tensor[i, i, Nr - r + i] = 1.0

# Now symmetrize PQ
for j in range(Nr):
    PQ_tensor[:, :, j] = 0.5 * (PQ_tensor[:, :, j] + PQ_tensor[:, :, j].T)
    
library_functions = [lambda x:x, lambda x, y:x*y, lambda x:x**2]  #, lambda ignored: 1]
library_function_names = [lambda x:x, lambda x, y:x+y, lambda x:x+x]  #, lambda ignored: 1]
sindy_library = ps.CustomLibrary(library_functions=library_functions,
                                 function_names=library_function_names)

In [ ]:
eta = 1e3
threshold = 0.0
gamma = 1e-4 * eta
beta = 0.9 * eta
sindy_opt = ps.clSR3(threshold=threshold, eta=eta, alpha_m=gamma, alpha_A=beta, vtol=1e-9,
                            PL=PL_tensor, PQ=PQ_tensor, max_iter=50, tol=1e-12, accel=False,
                            thresholder="l1", eigmin=-1e100, eigmax=-0.01, 
                            constraint_lhs=constraint_matrix,
                            constraint_rhs=constraint_zeros,
                            constraint_order="feature")
model = ps.SINDy(
            optimizer=sindy_opt,
            feature_library=sindy_library,
            differentiation_method=ps.FiniteDifference(drop_endpoints=True))
model.fit(a[:, :r], t=t)
model.print()
Xi = model.coefficients().T

# Check if identified model is stable
opt_m = sindy_opt.m_history_[-1]
mPQ = np.zeros(PL_tensor.shape)
for i in range(r):
    for j in range(i+1, r):
        mPQ[i, j, :, int((i+1)/2.0*(2*r-i)) + j - 1 - i] = opt_m
for i in range(r):
    mPQ[i, i, :, Nr - r + i] = opt_m
for i in range(r):
    for j in range(Nr):
        mPQ[:, :, i, j] = 0.5 * (mPQ[:, :, i, j] + mPQ[:, :, i, j].T)     
P_tensor = PL_tensor - mPQ
As = np.tensordot(P_tensor, Xi, axes=([3, 2], [0, 1]))
        #print('As: ', As)
eigvals, eigvecs = np.linalg.eig(As)
print(opt_m)
print('As eigvals: ', np.sort(eigvals))
print(np.all(eigvals < 0))

# Integrate model and plot results
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-20
integrator_keywords['h0'] = 1e-5
a_sindy3 = model.simulate(a[300,:r], t_sim, integrator_kws=integrator_keywords)

plt.figure(figsize=(10, 3))
plt.plot(t_sim, a_sindy3[:, [0, 1, -1]])
plt.plot([t_sim[0], t_sim[-1]], [a_max, a_max], 'k--')
plt.plot([t_sim[0], t_sim[-1]], [-a_max, -a_max], 'k--')
plt.ylim([-3, 3])
plt.xlim([0, 300])
plt.title('SINDy model')
plt.grid()
plt.figure()
plt.semilogy(sindy_opt.objective_history[2:])

plt.figure(figsize=(6, 6))
plt.subplot(231)
plt.plot(a[:, 0], a[:, 1])
plt.plot(a_sindy3[:, 0], a_sindy3[:, 1],'--')
plt.xlabel(r'$a_1$')
plt.ylabel(r'$a_2$')
plt.grid()

plt.subplot(232)
plt.plot(a[:, 0], a[:, -1])
plt.plot(a_sindy3[:, 0], a_sindy3[:, -1],'--')
plt.xlabel(r'$a_1$')
plt.ylabel(r'$a_\Delta$')
plt.grid()

plt.subplot(233)
plt.plot(a[:, 1], a[:, -1])
plt.plot(a_sindy3[:, 1], a_sindy3[:, -1],'--')
plt.xlabel(r'$a_1$')
plt.ylabel(r'$a_3$')
plt.grid()

plt.subplot(234)
plt.plot(a[:, 0], a[:, 4])
plt.plot(a_sindy3[:, 0], a_sindy3[:, 4],'--')
plt.xlabel(r'$a_1$')
plt.ylabel(r'$a_5$')
plt.grid()

plt.subplot(235)
plt.plot(a[:, 0], a[:, 5])
plt.plot(a_sindy3[:, 0], a_sindy3[:, 5],'--')
plt.xlabel(r'$a_1$')
plt.ylabel(r'$a_6$')
plt.grid()

plt.subplot(236)
plt.plot(a[:, 0], a[:, 6])
plt.plot(a_sindy3[:, 0], a_sindy3[:, 6],'--')
plt.xlabel(r'$a_1$')
plt.ylabel(r'$a_7$')
plt.grid()
plt.show()

Ls = np.tensordot(PL_tensor, Xi, axes=([3,2],[0,1]))
Q = np.tensordot(PQ_tensor, Xi, axes=([2],[0]))

0.004107689720373733 0.0 0.0
0.3622691147784462 0.0005454225683709152 0.0
0.3622691147784462 0.0005052293500970577 0.0
0.3622691147784462 0.00047174994607254175 0.0
0.3622691147784462 0.00044301037237097767 0.0
0.3622691147784462 0.00041775959853928944 0.0
0.3622691147784462 0.00039517942705337986 0.0
0.3622691147784462 0.0003747199915896739 0.0
0.3622691147784462 0.0003560018163976676 0.0
0.3622691147784462 0.00033875522730604534 0.0
0.3622691147784462 0.00032278201959338277 0.0
0.3622691147784462 0.00030793102665384054 0.0
0.3622691147784462 0.0002940825743345492 0.0
0.3622691147784462 0.0002811386254278375 0.0
0.3622691147784462 0.0002690165305472331 0.0
0.3622691147784462 0.00025764503352436127 0.0
0.3622691147784462 0.00024696167044352597 0.0
0.3622691147784462 0.00023691102572640126 0.0
0.3622691147784462 0.00022744351700520142 0.0
0.3622691147784462 0.00021851451061254164 0.0
0.3622691147784462 0.0002100836488190582 0.0
0.3622691147784462 0.00020211431741781586 0.0
0.36226911477

0.3622691147784462 8.437854752054592e-06 0.0
0.3622691147784462 8.382326944955801e-06 0.0
0.3622691147784462 8.328639636270432e-06 0.0
0.3622691147784462 8.276750703824026e-06 0.0
0.3622691147784462 8.226618501842402e-06 0.0
0.3622691147784462 8.178201831369958e-06 0.0
0.3622691147784462 8.131459913012095e-06 0.0
0.3622691147784462 8.086352362173805e-06 0.0
0.3622691147784462 8.04283916696475e-06 0.0
0.3622691147784462 8.000880668930172e-06 0.0
0.3622691147784462 7.960437546758284e-06 0.0
0.3622691147784462 7.921470803097228e-06 0.0
0.3622691147784462 7.883941754595164e-06 0.0
0.3622691147784462 7.847812025252657e-06 0.0
0.3622691147784462 7.8130435431482e-06 0.0
0.3622691147784462 7.77959854056589e-06 0.0
0.3622691147784462 7.74743955751831e-06 0.0
0.3622691147784462 7.716529448618973e-06 0.0
0.3622691147784462 7.686831393218292e-06 0.0
0.3622691147784462 7.658308908674451e-06 0.0
0.3622691147784462 7.63092586658816e-06 0.0
0.3622691147784462 7.604646511787064e-06 0.0
0.36226911477844

0.3622691147784462 7.05476999093859e-06 0.0
0.3622691147784462 7.0547627615068456e-06 0.0
0.3622691147784462 7.054755787962488e-06 0.0
0.3622691147784462 7.054749059403946e-06 0.0
0.3622691147784462 7.05474256545423e-06 0.0
0.3622691147784462 7.054736296233927e-06 0.0
0.3622691147784462 7.054730242335573e-06 0.0
0.3622691147784462 7.054724394799515e-06 0.0
0.3622691147784462 7.054718745090945e-06 0.0
0.3622691147784462 7.054713285078241e-06 0.0
0.3622691147784462 7.054708007012445e-06 0.0
0.3622691147784462 7.0547029035077925e-06 0.0
0.3622691147784462 7.054697967523355e-06 0.0
0.3622691147784462 7.054693192345559e-06 0.0
0.3622691147784462 7.05468857157173e-06 0.0
0.3622691147784462 7.054684099094385e-06 0.0
0.3622691147784462 7.054679769086486e-06 0.0
0.3622691147784462 7.054675575987376e-06 0.0
0.3622691147784462 7.054671514489505e-06 0.0
0.3622691147784462 7.0546675795257455e-06 0.0
0.3622691147784462 7.054663766257611e-06 0.0
0.3622691147784462 7.054660070063761e-06 0.0
0.36226911

0.3622691147784462 7.054523220387788e-06 0.0
0.3622691147784462 7.054523136250733e-06 0.0
0.3622691147784462 7.054523053879688e-06 0.0
0.3622691147784462 7.054522973237426e-06 0.0
0.3622691147784462 7.0545228942874874e-06 0.0
0.3622691147784462 7.054522816994192e-06 0.0
0.3622691147784462 7.054522741322644e-06 0.0
0.3622691147784462 7.0545226672386805e-06 0.0
0.3622691147784462 7.054522594708845e-06 0.0
0.3622691147784462 7.0545225237004315e-06 0.0
0.3622691147784462 7.0545224541813806e-06 0.0
0.3622691147784462 7.054522386120379e-06 0.0
0.3622691147784462 7.054522319486705e-06 0.0
0.3622691147784462 7.054522254250327e-06 0.0
0.3622691147784462 7.054522190381872e-06 0.0
0.3622691147784462 7.05452212785256e-06 0.0
0.3622691147784462 7.054522066634217e-06 0.0
0.3622691147784462 7.054522006699274e-06 0.0
0.3622691147784462 7.054521948020771e-06 0.0
0.3622691147784462 7.054521890572278e-06 0.0
0.3622691147784462 7.05452183432794e-06 0.0
0.3622691147784462 7.054521779262466e-06 0.0
0.362269

0.37107258557881084 6.625976874927946e-06 0.0
0.37107258557881084 6.61856333114837e-06 0.0
0.37107258557881084 6.611746479531288e-06 0.0
0.37107258557881084 6.605474344645969e-06 0.0
0.37107258557881084 6.599699707900277e-06 0.0
0.37107258557881084 6.594379643441178e-06 0.0
0.37107258557881084 6.589475104182334e-06 0.0
0.37107258557881084 6.584950551672366e-06 0.0
0.37107258557881084 6.580773624460671e-06 0.0
0.37107258557881084 6.5769148403934054e-06 0.0
0.37107258557881084 6.573347328913652e-06 0.0
0.37107258557881084 6.570046589974559e-06 0.0
0.37107258557881084 6.566990276622153e-06 0.0
0.37107258557881084 6.564157998682547e-06 0.0
0.37107258557881084 6.5615311453090505e-06 0.0
0.37107258557881084 6.559092724417215e-06 0.0
0.37107258557881084 6.556827217270789e-06 0.0
0.37107258557881084 6.5547204466824205e-06 0.0
0.37107258557881084 6.552759457467437e-06 0.0
0.37107258557881084 6.550932407940136e-06 0.0
0.37107258557881084 6.549228471374211e-06 0.0
0.37107258557881084 6.5476377464

0.37107258557881084 6.5124059651244275e-06 0.0
0.37107258557881084 6.512381528245662e-06 0.0
0.37107258557881084 6.512357533563091e-06 0.0
0.37107258557881084 6.512333973061013e-06 0.0
0.37107258557881084 6.512310838869855e-06 0.0
0.37107258557881084 6.512288123263359e-06 0.0
0.37107258557881084 6.5122658186560835e-06 0.0
0.37107258557881084 6.5122439176006066e-06 0.0
0.37107258557881084 6.512222412785084e-06 0.0
0.37107258557881084 6.512201297030709e-06 0.0
0.37107258557881084 6.512180563289273e-06 0.0
0.37107258557881084 6.512160204640716e-06 0.0
0.37107258557881084 6.512140214290792e-06 0.0
0.37107258557881084 6.51212058556878e-06 0.0
0.37107258557881084 6.5121013119251515e-06 0.0
0.37107258557881084 6.5120823869293715e-06 0.0
0.37107258557881084 6.512063804267724e-06 0.0
0.37107258557881084 6.512045557741159e-06 0.0
0.37107258557881084 6.512027641263162e-06 0.0
0.37107258557881084 6.512010048857771e-06 0.0
0.37107258557881084 6.511992774657439e-06 0.0
0.37107258557881084 6.51197581

0.37107258557881084 6.511107132486359e-06 0.0
0.37107258557881084 6.511106182177046e-06 0.0
0.37107258557881084 6.511105249037332e-06 0.0
0.37107258557881084 6.511104332756991e-06 0.0
0.37107258557881084 6.5111034330313625e-06 0.0
0.37107258557881084 6.511102549561323e-06 0.0
0.37107258557881084 6.511101682053099e-06 0.0
0.37107258557881084 6.51110083021826e-06 0.0
0.37107258557881084 6.511099993773611e-06 0.0
0.37107258557881084 6.511099172441036e-06 0.0
0.37107258557881084 6.511098365947493e-06 0.0
0.37107258557881084 6.511097574024804e-06 0.0
0.37107258557881084 6.511096796409705e-06 0.0
0.37107258557881084 6.511096032843653e-06 0.0
0.37107258557881084 6.511095283072799e-06 0.0
0.37107258557881084 6.511094546847858e-06 0.0
0.37107258557881084 6.511093823924072e-06 0.0
0.37107258557881084 6.51109311406111e-06 0.0
0.37107258557881084 6.511092417022949e-06 0.0
0.37107258557881084 6.5110917325778775e-06 0.0
0.37107258557881084 6.51109106049832e-06 0.0
0.37107258557881084 6.5110904005608

In [ ]:
# Energy of the models
E = np.sum(a**2, axis=1)
E_galerkin3 = np.sum(a_galerkin3**2, axis=1)
E_galerkin9 = np.sum(a_galerkin9**2, axis=1)
E_test = np.sum(a_sindy3**2, axis=1)

plt.figure(figsize=(12, 4))
plt.plot(t, E, 'k', label='DNS')
plt.plot(t_sim, E_test, 'm', r'SINDy (trapping quadratic)')
plt.plot(t_sim, E_galerkin3, label='POD-3')
plt.plot(t_sim, E_galerkin9, label='POD-9')

plt.legend(fontsize=14, loc=2)
plt.grid()
plt.xlim([0, 400])
plt.ylabel('Fluctuation energy')

In [ ]:
# Check some values
print(sindy_opt.PW_history_[-1])
print(sindy_opt.A_history_[-1])
print(sindy_opt.m_history_[-1])